In [2]:
import ccxt
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time
import os
from dotenv import load_dotenv


In [10]:
load_dotenv()

False

In [ ]:


print(date_start)
print(date_end)

In [19]:

class BinanceDataScraper:
    def __init__(self, api_key=None, api_secret=None):
        """
        Initialize Binance exchange connection
        
        Args:
            api_key (str, optional): Binance API key from .env
            api_secret (str, optional): Binance API secret from .env
        """
        # Fetch keys from environment variables if not provided
        api_key = api_key or os.getenv('KEY_1')
        api_secret = api_secret or os.getenv('KEY_2')
        
        # Initialize exchange
        self.exchange = ccxt.binance({
            'apiKey': api_key,
            'secret': api_secret,
            'enableRateLimit': True,
            'options': {
                'defaultType': 'spot'  # Use spot market by default
            }
        })

    def fetch_comprehensive_data(self, symbol):
        """
        Fetch comprehensive cryptocurrency data
        
        Args:
            symbol (str): Trading pair symbol (e.g., 'BTC/USDT')
            start_date (str): Start date in YYYY-MM-DD format
            end_date (str): End date in YYYY-MM-DD format
        
        Returns:
            pandas.DataFrame: Comprehensive cryptocurrency data
        """
        today = datetime.now()
        yest = today - timedelta(days=1)
        fifteen_days_ago = today - timedelta(days=15)

        # Format dates
        date_start = fifteen_days_ago.strftime("%Y-%m-%d")
        date_end = yest.strftime("%Y-%m-%d")

        # Convert dates to timestamps (in milliseconds)
        start_timestamp = int(datetime.strptime(date_start, '%Y-%m-%d').timestamp() * 1000)
        end_timestamp = int(datetime.strptime(date_end, '%Y-%m-%d').timestamp() * 1000)
        
        # Initialize empty list to store all OHLCV data
        ohlcv_data = []
        
        # Fetch data in chunks to avoid API limitations
        current_start = start_timestamp
        while current_start < end_timestamp:
            try:
                # Fetch 500 candles at a time (Binance limit)
                candles = self.exchange.fetch_ohlcv(
                    symbol, 
                    timeframe='1d', 
                    since=current_start,
                    limit=500
                )
                
                # Break if no more data
                if not candles:
                    break
                
                # Add to data list
                ohlcv_data.extend(candles)
                
                # Update start timestamp for next iteration
                current_start = candles[-1][0] + 1
                
                # Respect rate limits
                time.sleep(self.exchange.rateLimit / 1000)
                
            except Exception as e:
                print(f"Error fetching data for {symbol}: {e}")
                break
        
        # Convert to DataFrame
        df = pd.DataFrame(ohlcv_data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        
        # Convert timestamp to datetime and set as index
        df['date'] = pd.to_datetime(df['timestamp'], unit='ms')
        
        # Calculate market cap (estimated using close price and volume)
        # Note: This is a rough estimation and may not be entirely accurate
        df['market_cap'] = df['close'] * df['volume']
        
        # Compute daily returns
        df['daily_return'] = df['close'].pct_change()
        
        # Select and rename columns
        result_df = df[['date', 'close', 'volume', 'market_cap', 'daily_return']].copy()
        result_df.columns = ['date', 'price', 'volume', 'market_cap', 'daily_return']
        
        # Add asset column
        result_df['asset'] = symbol  # Use base currency as asset name
        
        # Set date as index
        result_df.set_index('date', inplace=True)
        
        # Filter date range
        result_df = result_df.loc[date_start:date_end]
        
        return result_df

    def fetch_top_symbols(self, limit=100):
        """
        Fetch top trading symbols by volume
        
        Args:
            limit (int): Number of top symbols to return
        
        Returns:
            list: Top trading symbols
        """
        try:
            # Load markets
            self.exchange.load_markets()
            
            # Sort markets by daily volume
            markets = sorted(
                self.exchange.markets.values(), 
                key=lambda x: x.get('quote', 'USDT') == 'USDT' and x.get('active', False),
                reverse=True
            )
            
            # Filter USDT pairs and get top symbols
            usdt_pairs = [
                market['symbol'] for market in markets 
                if market['quote'] == 'USDT' and market['active']
            ]
            
            return usdt_pairs[:limit]
        
        except Exception as e:
            print(f"Error fetching top symbols: {e}")
            return []

def main():
    # Initialize scraper using environment variables
    scraper = BinanceDataScraper()
    
    # Get top trading symbols
    top_symbols = scraper.fetch_top_symbols(limit=100)
    print(f"Fetching data for {len(top_symbols)} top symbols")
    
    # Dictionary to store all data
    all_data = {}
    
    # Fetch data for each symbol
    for symbol in top_symbols:
        try:
            print(f"Fetching data for {symbol}")
            df = scraper.fetch_comprehensive_data(symbol)
            
            if not df.empty:
                all_data[symbol] = df
                print(f"✓ Collected {len(df)} days of data for {symbol}")
            
            # Optional: Add a small delay between symbol fetches
            time.sleep(1)
        
        except Exception as e:
            print(f"Error processing {symbol}: {e}")
    
    # Combine all data
    if all_data:
        # Concatenate all dataframes
        market_df = pd.concat(all_data.values())
        
        print(f"\n✅ Data collection complete. Saved to market_df")
        print(f"Symbols collected: {len(all_data)}")
        print(f"Total records: {len(market_df)}")
    else:
        print("No data collected.")

if __name__ == "__main__":
    main()

Fetching data for 100 top symbols
Fetching data for BTC/USDT
✓ Collected 15 days of data for BTC/USDT
Fetching data for ETH/USDT
✓ Collected 15 days of data for ETH/USDT
Fetching data for BNB/USDT
✓ Collected 15 days of data for BNB/USDT
Fetching data for NEO/USDT
✓ Collected 15 days of data for NEO/USDT
Fetching data for LTC/USDT
✓ Collected 15 days of data for LTC/USDT
Fetching data for QTUM/USDT
✓ Collected 15 days of data for QTUM/USDT
Fetching data for ADA/USDT
✓ Collected 15 days of data for ADA/USDT
Fetching data for XRP/USDT
✓ Collected 15 days of data for XRP/USDT
Fetching data for EOS/USDT
✓ Collected 15 days of data for EOS/USDT
Fetching data for TUSD/USDT
✓ Collected 15 days of data for TUSD/USDT
Fetching data for IOTA/USDT
✓ Collected 15 days of data for IOTA/USDT
Fetching data for XLM/USDT
✓ Collected 15 days of data for XLM/USDT
Fetching data for ONT/USDT
✓ Collected 15 days of data for ONT/USDT
Fetching data for TRX/USDT
✓ Collected 15 days of data for TRX/USDT
Fetching